## NATASHA: SLOVNET И NAVEC... LOVE :-)

In [ ]:
# !pip install wikipedia
#!cd 
# текущий каталог загрузки - папка с блокнотами
#!chcp 65001
# 25 mb - этот через notebook почему-то тупит при загрузке
#!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
# 2mb
#!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar   
#!wget  https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_syntax_news_v1.tar 
    

In [56]:
import wikipedia

wikipedia.set_lang("ru")
moskow = wikipedia.page("Москва")
text = moskow.content

In [86]:
from navec import Navec
from slovnet import NER, Syntax, Morph
from razdel import sentenize, tokenize
from ipymarkup import show_span_ascii_markup
from ipymarkup import show_dep_ascii_markup


## NER распознавание именованных сущностей

In [85]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

#text = "погода в городе Москва"
markup = ner(text)

# print(type(markup))# slovnet.markup.SpanMarkup
# print(dir(markup))  # markup.spans

In [69]:
from collections import defaultdict

ENTITIES = defaultdict(set)
for span in markup.spans:
    ENTITIES[span.type].add(text[slice(span.start,span.stop)])

In [58]:
#ENTITIES

In [59]:
print(ENTITIES.keys())

dict_keys(['LOC', 'ORG', 'PER'])


In [65]:
print(list(ENTITIES["ORG"])[:20])  # первые 20 сущностей

['Театр Петра Фоменко', 'Банком Москвы', 'ГУМа', 'Москва', 'Экспериментаниум', 'НИИ;\nТушинский машиностроительный завод', 'Национальным центром', 'Московской железной дороге', 'ССВ', 'Известия', 'Алмаз', 'Торпедо', 'Всероссийское государственное совещание', 'Московским государственным университетом им. М. В. Ломоносова', 'ЦСКА', 'Департамент транспорта', 'Центральный аэрогидродинамический институт им. Жуковского (ЦАГИ)', 'Советской России', 'Румянцевский музей', 'Научная библиотека']


In [66]:
print(list(ENTITIES["LOC"])[:20])


['Солнечногорским районом', 'Садовое', 'Воскресенские ворота', 'Воздвиженке', 'Большой Дмитровке', 'Николо-Архангельский', 'Арх Москва', 'Нью-Йорке', 'Сингапуром', 'Красносельский', 'Москву', 'Серебряном Бору', 'Восточно-Европейской', 'Южнопортовый', 'Зеленоградский административный округ', 'Ростовом-на-Дону', 'Москворецко-Окской равнины', 'Курского', 'Можайское княжества', 'Сетунь']


In [67]:
print(list(ENTITIES["PER"])[:20])

['Вахитов', 'Святославом Ольговичем', 'Иване III', 'М. И. Старая Москва', 'Юрий Долгорукий', 'Физико-географическая', 'Отв', 'Данииле Александровиче', 'Марка Самойловича Лисянского', 'Владимирский', 'Ф. А. Брокгауза', 'Юрия Даниловича', 'Петре II', 'Григорием Неуйминым', 'Исаака Осиповича Дунаевского', 'Ивана Заруцкого', 'А. Н. Пахмутова', 'В. А. Садовничий', 'Сергея Ивановича Аграняна', 'См']


## сегментация и токенизация

In [74]:
chunks = []
for sent in sentenize(text):
    tokens = [_.text for _ in tokenize(sent.text)]
    chunks.append(tokens)

for sent in chunks[:10]:
    print(sent)

['Москва́', '(', 'произношение', ')', '—', 'столица', 'России', ',', 'город', 'федерального', 'значения', ',', 'административный', 'центр', 'Центрального', 'федерального', 'округа', 'и', 'центр', 'Московской', 'области', ',', 'в', 'состав', 'которой', 'не', 'входит', '.']
['Крупнейший', 'по', 'численности', 'населения', 'город', 'России', 'и', 'её', 'субъект', '—', '12', '655', '050', 'человек', '(', '2021', ')', ',', 'самый', 'населённый', 'из', 'городов', ',', 'полностью', 'расположенных', 'в', 'Европе', ',', 'входит', 'в', 'десятку', 'городов', 'мира', 'по', 'численности', 'населения', ',', 'крупнейший', 'русскоязычный', 'город', 'в', 'мире', '.']
['Центр', 'Московской', 'городской', 'агломерации', '.']
['Историческая', 'столица', 'Великого', 'княжества', 'Московского', ',', 'Русского', 'царства', ',', 'Российской', 'империи', '(', 'в', '1728—1730', 'годах', ')', ',', 'Советской', 'России', 'и', 'СССР', '.']
['Город-герой', '.']
['В', 'Москве', 'находятся', 'федеральные', 'органы', 

## syntax

In [83]:
syntax = Syntax.load('slovnet_syntax_news_v1.tar')
syntax.navec(navec)
markup = next(syntax.map(chunks[:1]))

# Convert CoNLL-style format to source, target indices
words, deps = [], []
for token in markup.tokens:
    words.append(token.text)
    source = int(token.head_id) - 1
    target = int(token.id) - 1
    if source > 0 and source != target:  # skip root, loops
        deps.append([source, target, token.rel])

show_dep_ascii_markup(words, deps)

        ┌────► Москва́          nsubj
        │   ┌► (                punct
        │ ┌─└─ произношение     
        │ └──► )                punct
        │   ┌► —                punct
┌─┌─┌─┌─└─┌─└─ столица          
│ │ │ │ │ └──► России           nmod
│ │ │ │ │   ┌► ,                punct
│ │ │ │ └►┌─└─ город            conj
│ │ │ │   │ ┌► федерального     amod
│ │ │ │   └►└─ значения         nmod
│ │ │ │   ┌──► ,                punct
│ │ │ │   │ ┌► административный amod
│ │ │ └►┌─└─└─ центр            conj
│ │ │   │ ┌──► Центрального     amod
│ │ │   │ │ ┌► федерального     amod
│ │ │   └►└─└─ округа           nmod
│ │ │       ┌► и                cc
│ │ └────►┌─└─ центр            conj
│ │       │ ┌► Московской       amod
│ │       └►└─ области          nmod
│ │   ┌──────► ,                punct
│ │   │     ┌► в                case
│ │   │ ┌►┌─└─ состав           obl
│ │   │ │ └──► которой          nmod
│ │   │ │   ┌► не               advmod
│ └──►└─└───└─ входит           acl:relc

## morfology

In [87]:
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)

markup = next(morph.map(chunks[:1]))
for token in markup.tokens:
    print(f'{token.text:>20} {token.tag}')

             Москва́ PROPN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
                   ( PUNCT
        произношение NOUN|Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing
                   ) PUNCT
                   — PUNCT
             столица NOUN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
              России PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
                   , PUNCT
               город NOUN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
        федерального ADJ|Case=Gen|Degree=Pos|Gender=Neut|Number=Sing
            значения NOUN|Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
                   , PUNCT
    административный ADJ|Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
               центр NOUN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
        Центрального ADJ|Case=Gen|Degree=Pos|Gender=Masc|Number=Sing
        федерального ADJ|Case=Gen|Degree=Pos|Gender=Masc|Number=Sing
              округа NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
             